# Create a Project
Note:  This requires that you have enabled products within SageMaker Studio

![](img/enable-service-catalog-portfolio-for-studio.png)

In [ ]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
sc = boto3.Session().client('servicecatalog', region_name=region)

In [ ]:
import time
timestamp = str(int(time.time() * 10**7))

In [ ]:
search_response = sc.search_products(
   Filters={
       'FullTextSearch': 
       [
           'MLOps template for model building, training, and deployment'
       ]
   }
)

sagemaker_pipeline_product_id = search_response['ProductViewSummaries'][0]['ProductId']
print(sagemaker_pipeline_product_id)

# pprint(search_response)

In [ ]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response['ProvisioningArtifacts'][0]['Id']

pprint(describe_response)

In [ ]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

In [ ]:
sagemaker_project_name = 'dsoaws-{}'.format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription='dsoaws-{}'.format(timestamp),
    ServiceCatalogProvisioningDetails={
        'ProductId': sagemaker_pipeline_product_id,
        'ProvisioningArtifactId': sagemaker_pipeline_product_provisioning_artifact_id
#        'PathId': 'string',
#         'ProvisioningParameters': [
#             {
#                 'Key': 'string',
#                 'Value': 'string'
#             },
#         ]
     }
#     Tags=[
#         {
#             'Key': 'string',
#             'Value': 'string'
#         },
#     ]
)


sagemaker_project_id = create_response['ProjectId']
sagemaker_project_arn = create_response['ProjectArn']

pprint(create_response)

In [ ]:
sagemaker_project_name_and_id = '{}-{}'.format(sagemaker_project_name, sagemaker_project_id)

print(sagemaker_project_name_and_id)

# Clone Repos

In [ ]:
print(region)

In [ ]:
%%bash

# git clone https://git-codecommit.$region.amazonaws.com/v1/repos/sagemaker-$sagemaker_project_name_and_id-modelbuild
# git clone https://git-codecommit.$region.amazonaws.com/v1/repos/sagemaker-$sagemaker_project_name_and_id-modeldeploy

# TODO:  Add run_pipeline.py and codebuild-buildspec.yaml
# TODO:  We have to change this to $HOME dir or something to keep it out of our workshop/ git project
git clone https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-$(sagemaker_project_name_and_id)-modelbuild ../../
git clone https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-$(sagemaker_project_name_and_id)-modeldeploy ../../

    
# TODO:  Add run_pipeline.py and codebuild-buildspec.yaml, and setup.py
# TODO:  Copy our ./project-dsoaws-p-ibxfjw9nuim7/ files into these repos
cp ./project-dsoaws-p-ibxfjw9nuim7/sagemaker-project-dsoaws-p-ibxfjw9nuim7-modeldeploy/staging-config.json ../../sagemaker-$(sagemaker_project_name_and_id)-modeldeploy/
cp ./project-dsoaws-p-ibxfjw9nuim7/sagemaker-project-dsoaws-p-ibxfjw9nuim7-modeldeploy/prod-config.json ./../sagemaker-$(sagemaker_project_name_and_id)-modeldeploy/

In [ ]:
# TODO:  Commit this code back to CodeCommit (this might require more permissions and/or switching to git clone git://?)
%%bash

cd ../../sagemaker-dsoaws-16086664484376606-p-47xljwgej1p2-modeldeploy/ 

git status

git add --all .

git commit -m "updated instance types"

git push

In [ ]:
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

In [ ]:
%store